In [1]:
import sys
import re
import sqlite3
import pandas as pd
sys.path.append('../src/')
from scraper.scraper import Scraper
from results.results import Results
from database.create_db import Database
from database.database import Event, Race, Results


In [29]:
import importlib
import database.database
importlib.reload(database.database)
from database.database import Event, Race, Results


In [2]:
db: Database = Database.create_database(path='../data/min_data.db')


Database created successfully.


In [3]:
scraper = Scraper()
events = scraper.getEvents()
years = scraper.getEventsYears()


In [4]:
  # Get the list of events and years
events = dict(sorted(scraper.getEvents().items(), key=lambda item: item[1]))
# Remove years and strip
events = {key: ' '.join(word for word in value.split() if not word.isdigit() or len(word) != 4).strip() for key, value in events.items()}
events = {key: re.sub(r'^\d{4}|\d{4}$', '', value).strip() for key, value in events.items()}
# Remove French ordinals
events = {key: re.sub(r'(\d{1,2}(?:e|ème))', '', value).strip() for key, value in events.items()}
# Remove HTML tags
events = {key: re.sub(r'<[^<]+?>', '', value).strip() for key, value in events.items()}
# Sort alphabetically
events = dict(sorted(events.items(), key=lambda item: item[1]))


In [5]:
for code, name in events.items():
    if code in years:
        for year in years[code]: 
            event = Event(event_code=code,
                        event_name=name,
                        year=year,
                        country=None,
                        db=db)
            event.save_to_database()

In [6]:
events_min = {'pordondecaminaeloso': '100 MILLAS POR DONDE CAMINA EL OSO',
                                'serracapicorsu': 'A Serra di u Capicorsu',
                                 'saintelyon': 'ASICS SAINTÉLYON'}

In [30]:
for event, name in events_min.items():
    if event in years:
        for year in years[event]:
            scraper.setEvents([event])
            scraper.setYears([year])
            races = scraper.getRaces()
            if event not in races:
                #st.write(f'No data available for {events[event]} {year}. Please select another event.')
                pass
            elif year not in races[event]:
                #st.write(f'No data available for {events[event]} {year}. Please select another event or year.')
                pass
            else:
                races = races[event][year]
                print(races)
                for code, name in races.items():
                    event_id= Event.get_id_from_code_year(event, year, db)
                    #print(code, event_id)
                    scraper.setEvents([event])
                    scraper.setYears([year])
                    scraper.setRace(code)
                    try:
                        # try needed because bib number = 2 is not always true
                        # for final version results are downloaded and bibN is taken as
                        # bibN=raw_results.iloc[0]['doss']
                        race_info = scraper.getRaceInfo(bibN=2)
                        control_points = scraper.getControlPoints()[code]
                        race_data = control_points.popitem()[1]
                        # {'date': '2023-02-25', 'tz': '1', 'hd': '00:00:13', 'jd': '6'}
                        r = Race(race_id=code, event_id=event_id, race_name=name, distance=race_data[0],
                        elevation_pos=race_data[1], elevation_neg=race_data[2], departure_datetime=' '.join([race_info['date'], race_info['hd']]),
                        results_filepath=None, db=db)
                        r.save_to_database()
                    except:
                        pass

{'100m': '100 MILLAS', '117km': '117 KM', '65km': '65 KM', '46km': '46 KM'}


/VSCode/personnel/MLTrail/src/scraper/scraper.py:254: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(data, 'lxml')


{'solo': 'SOLO - A Serra di u Capicorsu', 'relais': 'RELAIS - A Serra di u Capicorsu'}
{'78km': 'SAINTÉLYON', '156km': 'LYONSAINTÉLYON', 'rel2': 'SAINTÉLYON RELAIS 2', 'rel3': 'SAINTÉLYON RELAIS 3', 'rel4': 'SAINTÉLYON RELAIS 4', '44km': 'SAINTÉXPRESS', '24km': 'SAINTÉSPRINT', '13km': 'SAINTÉTIC'}
{'78km': 'SAINTÉLYON', 'rel2': 'SAINTÉLYON RELAIS 2', 'rel3': 'SAINTÉLYON RELAIS 3', 'rel4': 'SAINTÉLYON RELAIS 4', '44km': 'SAINTÉXPRESS', '156km': 'LYONSAINTÉLYON', '24km': 'SAINTÉSPRINT', '13km': 'SAINTÉTIC'}
{'78km': 'SAINTÉLYON', '156km': 'LYONSAINTÉLYON', 'rel2': 'SAINTÉLYON RELAIS 2', 'rel3': 'SAINTÉLYON RELAIS 3', 'rel4': 'SAINTÉLYON RELAIS 4', '46km': 'SAINTÉXPRESS', '23km': 'SAINTÉSPRINT', '13km': 'SAINTÉTIC'}
{'76km': 'SAINTÉLYON', 'rel2': 'SAINTÉLYON RELAIS 2', 'rel3': 'SAINTÉLYON RELAIS 3', 'rel4': 'SAINTÉLYON RELAIS 4', '44km': 'SAINTÉXPRESS', '23km': 'SAINTÉSPRINT', '12km': 'SAINTÉTIC', '152km': 'LYONSAINTÉLYON'}
{'81km': 'SAINTÉLYON', 'rel2': 'SAINTÉLYON RELAIS 2', 'rel3': 'SA

- add getid to tests
- add update race case into tests
- add results download + load to DB


In [8]:
conn = sqlite3.connect(db.path)
with conn:
    cursor = conn.cursor()
    cursor.execute('''
        SELECT *
        FROM events
    ''')
    rows = cursor.fetchall()
conn.close()
for row in rows:
    print(row)

(1, 'pordondecaminaeloso', '100 MILLAS POR DONDE CAMINA EL OSO', '2021', None)
(2, 'serracapicorsu', 'A Serra di u Capicorsu', '2019', None)
(3, 'saintelyon', 'ASICS SAINTÉLYON', '2023', None)
(4, 'saintelyon', 'ASICS SAINTÉLYON', '2022', None)
(5, 'saintelyon', 'ASICS SAINTÉLYON', '2021', None)
(6, 'saintelyon', 'ASICS SAINTÉLYON', '2019', None)
(7, 'saintelyon', 'ASICS SAINTÉLYON', '2018', None)
(8, 'saintelyon', 'ASICS SAINTÉLYON', '2017', None)
(9, 'saintelyon', 'ASICS SAINTÉLYON', '2016', None)
(10, 'saintelyon', 'ASICS SAINTÉLYON', '2015', None)
(11, 'saintelyon', 'ASICS SAINTÉLYON', '2014', None)
(12, 'saintelyon', 'ASICS SAINTÉLYON', '2013', None)
(13, 'amazeanjungletrail', 'Amazean Jungle Thailand by UTMB', '2023', None)
(14, 'amazeanjungletrail', 'Amazean Jungle Thailand by UTMB', '2022', None)
(15, 'bk-bidarray', 'BK - Bidarray', '2023', None)
(16, 'bk-bidarray', 'BK - Bidarray', '2022', None)
(17, 'bk-bidarray', 'BK - Bidarray', '2019', None)
(18, 'bk-bidarray', 'BK - Bidar

In [31]:
conn = sqlite3.connect(db.path)
with conn:
    cursor = conn.cursor()
    cursor.execute('''
        SELECT *
        FROM races
        LIMIT 25
    ''')
    rows = cursor.fetchall()
conn.close()
for row in rows:
    print(row)

('100m', 1, '100 MILLAS', 174.2, 9715, -9715, '2021-09-17 16:04:30', None)
('117km', 1, '117 KM', 117.2, 6712, -6639, '2021-09-17 16:04:30', None)
('65km', 1, '65 KM', 64.22, 3557, -4027, '2021-09-17 16:04:30', None)
('46km', 1, '46 KM', 45.67, 1818, -2969, '2021-09-17 16:04:30', None)
('solo', 2, 'SOLO - A Serra di u Capicorsu', 57.23, 2976, -2976, '2019-10-19 06:04:02', None)
('relais', 2, 'RELAIS - A Serra di u Capicorsu', 57.23, 2976, -2976, '2019-10-19 06:04:02', None)
('78km', 3, 'SAINTÉLYON', 78.28, 2215, -2542, '2023-12-02 23:41:32', None)
('156km', 3, 'LYONSAINTÉLYON', 142.14, 4478, -4333, '2023-12-02 23:41:32', None)
('rel2', 3, 'SAINTÉLYON RELAIS 2', 78.28, 2215, -2542, '2023-12-02 23:41:32', None)
('rel3', 3, 'SAINTÉLYON RELAIS 3', 78.28, 2215, -2542, '2023-12-02 23:41:32', None)
('rel4', 3, 'SAINTÉLYON RELAIS 4', 78.28, 2215, -2542, '2023-12-02 23:41:32', None)
('44km', 3, 'SAINTÉXPRESS', 44.44, 1063, -1591, '2023-12-02 23:41:32', None)
('24km', 3, 'SAINTÉSPRINT', 23.73, 4

In [26]:
conn = sqlite3.connect(db.path)
with conn:
    cursor = conn.cursor()
    cursor.execute('''
        DELETE 
        FROM races
    ''')
    conn.commit()
    rows = cursor.fetchall()
    cursor.execute('''
        SELECT *
        FROM races
    ''')
conn.close()
for row in rows:
    print(row)